In [2]:
#r, Q continuous 

# /Users/arshiabansal/Desktop/product-1211-sales-order-data.csv

In [3]:
import pandas as pd
import numpy as np

def r_Q_continuous(
    sales_order_file_path,
    initial_inventory=100000,
    holding_cost=1,
    ordering_cost=50,
    desired_service_level=0.95,
    max_shelf_space_for_SKU=124400,
    MOQ=1000,
    today_date='2021-04-01',
    lead_time=4,
    back_orders=50,
    open_orders=100,
    total_days=100,
    review_period=1,
    num_simulations=1000
):
    # Read the sales order data
    demand_data = pd.read_csv(sales_order_file_path, index_col=0, parse_dates=True)

    # Rename the column if necessary
    if 'total qty' in demand_data.columns:
        demand_data.rename(columns={'total qty': 'total_qty'}, inplace=True)

    # Convert 'total_qty' to numeric, coercing errors to NaN
    demand_data['total_qty'] = pd.to_numeric(demand_data['total_qty'], errors='coerce')

    # Remove rows with NaN values in 'total_qty'
    demand_data.dropna(subset=['total_qty'], inplace=True)

    # Helper functions
    def safety_stock_calculation(demand_std_dev, lead_time, Z):
        Z_score = abs(np.percentile(np.random.normal(size=1000000), 100 * Z))
        return Z_score * (lead_time ** 0.5) * demand_std_dev

    def forecasted_demand_over_lead_time(demand_data, today_date, lead_time):
        todays_index = demand_data.index.get_loc(today_date)
        start_index = todays_index + 1
        end_index = todays_index + lead_time
        lead_time_df = demand_data.iloc[start_index:end_index]
        return lead_time_df['total_qty'].sum()

    def forecasted_demand_till_next_review(demand_data, today_date, review_period):
        todays_index = demand_data.index.get_loc(today_date)
        start_index = todays_index
        end_index = todays_index + review_period
        review_period_df = demand_data.iloc[start_index:end_index]
        return review_period_df['total_qty'].sum()

    def calculate_forecasted_demand_std(demand_data, today_date, review_period):
        todays_index = demand_data.index.get_loc(today_date)
        start_index = todays_index
        end_index = todays_index + review_period
        review_period_df = demand_data.iloc[start_index:end_index]
        
        if len(review_period_df) < 2:
            return 0.0
        
        return review_period_df['total_qty'].std()

    # Add debug print statements
    dd_till_next_review = forecasted_demand_till_next_review(demand_data, today_date, lead_time)
    forecasted_demand_mean = dd_till_next_review / lead_time
    forecasted_demand_std = calculate_forecasted_demand_std(demand_data, today_date, lead_time)
    print("Forecasted Demand Mean:", forecasted_demand_mean)
    print("Forecasted Demand Std Dev:", forecasted_demand_std)

    safety_stock = safety_stock_calculation(forecasted_demand_std, lead_time, desired_service_level)
    dd_over_lead_time = forecasted_demand_over_lead_time(demand_data, today_date, lead_time)
    reorder_point = safety_stock + dd_over_lead_time
    print("Reorder Point:", reorder_point)

    def calculate_total_cost(inventory_levels, order_placed, holding_cost, ordering_cost):
        average_inventory = np.mean(inventory_levels)
        total_orders = np.sum(order_placed > 0)
        total_holding_cost = average_inventory * holding_cost
        total_ordering_cost = total_orders * ordering_cost
        return total_holding_cost + total_ordering_cost

    # Monte Carlo Simulation
    def monte_carlo_optimal_Q_cost(forecasted_demand_mean, forecasted_demand_std, lead_time, MOQ, max_shelf_space, initial_inventory, back_orders, open_orders, holding_cost, ordering_cost, review_period=1, num_simulations=1000):
        total_costs = []
        optimal_Qs = []

        for Q in range(MOQ, max_shelf_space, MOQ):
            costs = []
            
            for sim in range(num_simulations):
                daily_demand = np.random.normal(forecasted_demand_mean, forecasted_demand_std, total_days)
                inventory_level = initial_inventory
                total_ordering_cost = 0
                total_holding_cost = 0

                for day in range(total_days):
                    if inventory_level < reorder_point:
                        order_quantity = Q
                        if order_quantity > max_shelf_space:
                            order_quantity = max_shelf_space
                        total_ordering_cost += ordering_cost
                        inventory_level += order_quantity - daily_demand[day]
                    else:
                        inventory_level -= daily_demand[day]

                    total_holding_cost += inventory_level * holding_cost

                total_cost = total_ordering_cost + total_holding_cost
                costs.append(total_cost)
            
            average_cost = np.mean(costs)
            total_costs.append(average_cost)
            optimal_Qs.append(Q)

        min_cost_index = np.argmin(total_costs)
        optimal_Q_min_cost = optimal_Qs[min_cost_index]
        min_total_cost = total_costs[min_cost_index]

        return optimal_Q_min_cost, min_total_cost

    optimal_Q_min_cost, min_total_cost = monte_carlo_optimal_Q_cost(
        forecasted_demand_mean,
        forecasted_demand_std,
        lead_time,
        MOQ,
        max_shelf_space_for_SKU,
        initial_inventory,
        back_orders,
        open_orders,
        holding_cost,
        ordering_cost
    )

    print("Optimal Order Quantity (Q) to Minimize Cost:", optimal_Q_min_cost)
    print("Minimum Total Cost:", min_total_cost)

    # Inventory System Simulation
    try:
        todays_index = demand_data.index.get_loc(today_date)
    except KeyError:
        print(f"Date {today_date} not found in demand data index.")
        todays_index = 0  # Assuming we start from the beginning if date not found

    end_index = todays_index + total_days

    forecasted_demand = demand_data.iloc[todays_index:end_index]['total_qty'].values
    days = pd.date_range(start=today_date, periods=total_days)

    data = {
        'day_tracker': days,
        'stock_start': np.zeros(total_days),
        'forecasted_demand': forecasted_demand,
        'stock_end': np.zeros(total_days),
        'inventory_check': [False] * total_days,
        'order_placed': [False] * total_days,
        'orders_in_transit': [0] * total_days  # Track orders in transit
    }

    df = pd.DataFrame(data)
    df.at[0, 'stock_start'] = initial_inventory - back_orders - open_orders

    for i in range(total_days):
        if i > 0:
            df.at[i, 'stock_start'] = df.at[i - 1, 'stock_end'] + (df.at[i - lead_time, 'orders_in_transit'] if i - lead_time >= 0 else 0)

        df.at[i, 'stock_end'] = df.at[i, 'stock_start'] - df.at[i, 'forecasted_demand']

        if df.at[i, 'stock_end'] < 0:
            df.at[i, 'stock_end'] = 0

        if (i + 1) % review_period == 0:
            df.at[i, 'inventory_check'] = True

            if df.at[i, 'stock_end'] < reorder_point:
                df.at[i, 'order_placed'] = True
                df.at[i, 'orders_in_transit'] = optimal_Q_min_cost

    return df, optimal_Q_min_cost, min_total_cost

# Example usage:
if __name__ == "__main__":
    sales_order_file_path = input("Please enter the path to the sales order data file: ")
    result_df, optimal_Q, min_cost = r_Q_continuous(sales_order_file_path)
    print(result_df)
    print("Optimal Order Quantity (Q) to Minimize Cost:", optimal_Q)
    print("Minimum Total Cost:", min_cost)


/var/folders/ns/1yxzfx892f36qy2_byrlq2kw0000gn/T/ipykernel_70598/346773475.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  demand_data = pd.read_csv(sales_order_file_path, index_col=0, parse_dates=True)


Forecasted Demand Mean: 49.75
Forecasted Demand Std Dev: 19.619293225462194
Reorder Point: 184.59435433870289
Optimal Order Quantity (Q) to Minimize Cost: 5000
Minimum Total Cost: 9747771.48866245
   day_tracker  stock_start  forecasted_demand  stock_end  inventory_check  \
0   2021-04-01      99850.0                 79    99771.0             True   
1   2021-04-02      99771.0                 37    99734.0             True   
2   2021-04-03      99734.0                 42    99692.0             True   
3   2021-04-04      99692.0                 41    99651.0             True   
4   2021-04-05      99651.0                 44    99607.0             True   
..         ...          ...                ...        ...              ...   
95  2021-07-05      95093.0                 61    95032.0             True   
96  2021-07-06      95032.0                134    94898.0             True   
97  2021-07-07      94898.0                  4    94894.0             True   
98  2021-07-08      948